# Week 05 — Correlation vs Causation (상관/인과/혼란변수)

이 노트북은 **실전 통계 → 해석 → 비즈니스 액션**을 훈련하기 위한 템플릿입니다.
각 섹션의 ✅ 체크포인트를 채우면서 진행하세요.


## 1) Business Questions
- 마케팅비가 매출을 '올린다'고 말해도 되는가?
- 채널/세그먼트가 섞여 생기는 착시(Simpson's paradox)는 없는가?

## 2) Why this matters (Theory)
- 상관관계는 인과가 아님 (confounder를 통제해야 함)
- 세그먼트 혼합이 전체 상관을 뒤집을 수 있음
- 의사결정은 실험 또는 통제된 분석과 함께

## 3) Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

DATA_PATH = "data/sales_marketing_example_2025.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"])

# Quick sanity checks
display(df.head())
print("rows, cols:", df.shape)
print("missing rate (top):")
display(df.isna().mean().sort_values(ascending=False).head(10))

## 4) Helper Functions

In [ ]:
def one_sentence_insight(prompt: str):
    """Write your one-sentence business insight below."""
    print("\n🧾 One-sentence insight (fill this):")
    print(" -", prompt)

def bootstrap_mean_diff(x, y, iters=5000, seed=0):
    """Bootstrap CI for difference in means (x - y)."""
    r = np.random.default_rng(seed)
    x = np.asarray(pd.Series(x).fillna(0))
    y = np.asarray(pd.Series(y).fillna(0))
    diffs = []
    for _ in range(iters):
        xs = r.choice(x, size=len(x), replace=True)
        ys = r.choice(y, size=len(y), replace=True)
        diffs.append(xs.mean() - ys.mean())
    diffs = np.array(diffs)
    return diffs.mean(), np.quantile(diffs, [0.025, 0.975])

def rate_ci_wilson(successes, n, alpha=0.05):
    """Wilson score interval for a proportion."""
    if n == 0:
        return (np.nan, np.nan)
    z = stats.norm.ppf(1 - alpha/2)
    phat = successes / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n)) / denom
    half = z * np.sqrt((phat*(1-phat) + z**2/(4*n)) / n) / denom
    return center - half, center + half

---
## 5) Analysis

### 5.1 Correlation matrix (key numeric features)
✅ 상관을 보고 바로 결론 내리지 말고, 다음 단계(통제)를 준비합니다.

In [ ]:
num = df[['marketing_spend','impressions','clicks','leads','sales_calls','discount_rate','revenue','gross_margin']].copy()
corr = num.corr(numeric_only=True)
display(corr)

In [ ]:
one_sentence_insight('가장 강한 상관 1~2개를 찾고, 인과로 착각할 위험을 한 문장')

### 5.2 Stratified view: spend vs revenue by channel
✅ 채널별로 관계가 달라지는지 확인합니다.

In [ ]:
tmp = df.dropna(subset=['marketing_spend']).copy()
tmp['log_spend'] = np.log1p(tmp['marketing_spend'])
tmp['log_revenue'] = np.log1p(tmp['revenue'].fillna(0))

for ch, sub in tmp.groupby('channel'):
    plt.figure()
    plt.scatter(sub['log_spend'], sub['log_revenue'], s=5)
    plt.title(f"log(spend) vs log(revenue) — {ch}")
    plt.xlabel("log_spend")
    plt.ylabel("log_revenue")
    plt.show()

In [ ]:
one_sentence_insight('전체 상관과 채널별 상관이 다르면 어떤 착시가 생길 수 있는지 한 문장')

---
## 6) Interpretation Checklist
- 결과가 '유의'하면, **효과 크기/CI**가 실무적으로 의미 있는가?
- 결과가 '비유의'하면, **표본 부족/측정오류/세그먼트 혼합** 가능성은?
- 액션을 한다면, **누구에게/어디에/얼마나** 적용할 것인가?



## Executive Summary (메일/슬라이드에 그대로 붙여넣기)

- **핵심 질문:**  
- **결론(한 문장):**  
- **근거 지표:** (효과크기 / p-value / CI 등)  
- **비즈니스 임팩트:** (예: 월 매출 +X, 마진 -Y 위험)  
- **권장 액션:** (실험/타겟 변경/채널 믹스 조정)  
- **추가 분석 필요:**  
